# Récupération des données pour l'étude départementale

## 1- Importation des librairies externes et classiques

In [ ]:
# Importation des librairies externes
!pip install git+https://github.com/InseeFrLab/Py-Insee-Data@master --ignore-install PyYAML
!pip install python-Levenshtein
!pip install openpyxl

In [ ]:
# Importation des librairies 
import requests
import pandas as pd
import pynsee
import pynsee.download
from pynsee.sirene import *
import time
from tqdm import tqdm
from pynsee.utils.init_conn import init_conn
import os
from zipfile import ZipFile, Path
import io

In [ ]:
# token API insee
init_conn(insee_key=os.environ['INSEE_KEY'], insee_secret=os.environ['INSEE_SECRET'])

## 2- Téléchargement des données de SIRENE

In [ ]:
### Téléchargement des données Sirene sur les établissements Siret de boucherie-charcuterie (code 4722Z) à l'instant T
# Téléchargement via Insee sirene data  by sirene.fr website

session = requests.session()
URL_ACCUEIL = "https://www.sirene.fr/sirene/public/creation-fichier"
URL_REQUETE_POST = "https://www.sirene.fr/sirene/public/verifier-creation-fichier.action"
URL_DEVIS="https://www.sirene.fr/sirene/client//sirene/client/devis-demande.action"

URL_RECAP="https://www.sirene.fr/sirene/client/telechargement-pret-fichier.action"
URL_TELECHARGEMENT_POST="https://www.sirene.fr/sirene/client/telechargement-fichier"
POST_DATA={
'fichier.caracteristiques.etat.actif': 'true',
'__checkbox_fichier.caracteristiques.etat.actif': 'true',
'__checkbox_fichier.caracteristiques.etat.ferme': 'true',
'fichier.caracteristiques.typeEtablissement': '1',
'fichier.caracteristiques.geoFile': 'false',
'fichier.caracteristiques.cjFile': 'false',
'fichier.caracteristiques.apeFile': 'false',
'fichier.caracteristiques.sirenSiretFile': 'false',
'fichiersCriteres.criteresGeographie.upload': '(binaire)',
'__multiselect_fichier.criteres.geographie.region.modalites': '',
'__multiselect_fichier.criteres.geographie.departement.modalites': '',
'fichiersCriteres.criteresActivite.upload': '(binaire)',
'__multiselect_fichier.criteres.activite.section.modalites': '',
'__multiselect_fichier.criteres.activite.division.modalites': '',
'__multiselect_fichier.criteres.activite.groupe.modalites': '',
'fichier.criteres.activite.sousClasse.modalites': '4722Z',
'__multiselect_fichier.criteres.activite.sousClasse.modalites': '',
'fichiersCriteres.criteresNature.upload': '(binaire)',
'__multiselect_fichier.criteres.nature.entrepriseNiveau1.modalites': '',
'__multiselect_fichier.criteres.nature.entrepriseNiveau2.modalites': '',
'__multiselect_fichier.criteres.nature.entrepriseNiveau3.modalites': '',
'__multiselect_fichier.criteres.trancheEffectif.modalites': '',
'fichier.criteres.dateCreation.date': '',
'fichier.criteres.dateCreation.dateFin': '',
'fichier.criteres.dateMaj.date': '',
'fichier.criteres.dateMaj.dateFin': '',
'fichiersCriteres.criteresSirenSiret.upload': '(binaire)',
'__multiselect_fichier.criteres.criteresCatEnt.modalites': '',
'strutsLocale': 'fr_FR',
}

result=session.get(URL_ACCUEIL)
result=session.post(URL_REQUETE_POST, data=POST_DATA)


result=session.get(URL_RECAP)

result=session.get(URL_DEVIS)

result=session.post(URL_TELECHARGEMENT_POST)

donnees_sirene_fr=pd.read_csv(io.StringIO(result.content.decode('utf-8')),dtype=str)


In [ ]:
donnees_sirene_fr.shape
donnees_sirene_fr.sample(10)

* les données de Sirene.fr sont des données d'entreprises France entière. Pour l'étude, le nombre d'établissement par département est considéré :

In [ ]:
# donnees_sirene_fr_agregees_par_departement = donnees_sirene_fr.groupby().sum()

## 3- Téléchargement des données du ministère de l'intérieur

In [ ]:
#Téléchargement des données de crimes et délits rapportés déclarés auprès des commissariats 
url_datagouv_min_interieur="https://www.data.gouv.fr/fr/datasets/r/d792092f-b1f7-4180-a367-d043200c1520"

donnees_police_nationale_2018=pd.read_excel(url_datagouv_min_interieur, sheet_name='Services PN 2018', skiprows=[1,2])
donnees_gendarmerie_nationale_2018=pd.read_excel(url_datagouv_min_interieur, sheet_name='Services GN 2018', skiprows=[1,2])


In [ ]:
donnees_police_nationale_2018.head(10)
donnees_gendarmerie_nationale_2018.head(10)

* Agrégation par département, transposition et fusion :

In [ ]:
# donnees_securite_interieur_agregees_par_departement = ...

## 4 - Téléchargement des revenus localisés

In [ ]:
#Téléchargement des données Filosofi au niveau IRIS (millénisme 2018)
URL_FILO_DECL_2018="https://www.insee.fr/fr/statistiques/fichier/5055909/BASE_TD_FILO_DEC_IRIS_2018.xlsx"
URL_FILO_DISP_2018="https://www.insee.fr/fr/statistiques/fichier/5055909/BASE_TD_FILO_DISP_IRIS_2018.xlsx"

donnees_filo_decl_2018=pd.read_excel(URL_FILO_DECL_2018, sheet_name='IRIS_DEC', skiprows=5)
donnees_filo_disp_2018=pd.read_excel(URL_FILO_DISP_2018, sheet_name='IRIS_DISP', skiprows=5)

In [ ]:
print(donnees_filo_decl_2018.head(10))
print(donnees_filo_disp_2018.head(10))

In [ ]:
URL_FILO_ZIP_2018="https://www.insee.fr/fr/statistiques/fichier/5009236/base-cc-filosofi-2018_CSV_geo2021.zip"
#Télécharger le zip de l'URL
r=requests.get(URL_FILO_ZIP_2018)

open("base-cc-filosofi-2018_CSV_geo2021.zip", 'wb').write(r.content)
    
    
with ZipFile("base-cc-filosofi-2018_CSV_geo2021.zip",'r') as myzip:
    data = myzip.open("cc_filosofi_2018_COM-geo2021.CSV")
    data2 = myzip.open("cc_filosofi_2018_DEP-geo2021.CSV")
    
donnees_filo_communes_2018=pd.read_csv(data,sep=';')    
donnees_filo_departement_2018=pd.read_csv(data2,sep=';')    

In [ ]:
donnees_filo_communes_2018.head(10)
donnees_filo_departement_2018.head(10)

## 5 - Téléchargement des populations légales

In [ ]:
#Téléchargement des populations légales du millénisme 2018 
URL_POP_LEG="https://www.insee.fr/fr/statistiques/fichier/4989724/ensemble.zip"

#Télécharger le zip de l'URL
r=requests.get(URL_POP_LEG)

open("ensemble.zip", 'wb').write(r.content)
    
    
with ZipFile("ensemble.zip",'r') as myzip:
    data = myzip.open("Departements.csv")

donnees_pop_leg_18=pd.read_csv(data,sep=';')     

In [ ]:
donnees_pop_leg_18.sample(10)

## 6 - Téléchargement des données d'emplois localisés

In [ ]:
#Téléchargement des données d'emploi localisées
donnees_emploi_statut_18 = pynsee.download.telechargerDonnees("ESTEL_T201", date = "2018")
donnees_emploi_statut_18.sample(10)

donnees_emploi_sexe_18 = pynsee.download.telechargerDonnees("ESTEL_T202", date = "2018")
donnees_emploi_sexe_18.sample(10)


URL_TAUX_CHOMAGE_PAR_DEP="https://www.insee.fr/fr/statistiques/fichier/2012804/sl_etc_2021T2.xls"


donnees_chomage_par_departement_series_longues=pd.read_excel(URL_TAUX_CHOMAGE_PAR_DEP, sheet_name='Département', skiprows=3)

In [ ]:
donnees_chomage_par_departement_series_longues.head(10)

## 7 - Téléchargement du Code Officiel Géographique

* Pour des raisons de simplicité, nous travaillons en géographie 2021 :

In [ ]:
#cog_commune = pynsee.download.telechargerDonnees("COG_COMMUNE", date = "dernier")
URL_COG_2021="https://www.insee.fr/fr/statistiques/fichier/5057840/commune2021-csv.zip"

#Télécharger le zip de l'URL
r=requests.get(URL_COG_2021)

open("commune2021-csv.zip", 'wb').write(r.content)
    
    
with ZipFile("commune2021-csv.zip",'r') as myzip:
    data = myzip.open("commune2021.csv")

donnees_cog_2021=pd.read_csv(data)     

In [ ]:
donnees_cog_2021

## 8 - Fusion des données

In [ ]:
# donnees_2018=join()